# Notebook 05: Reentrenamiento Mejorado

## Mejoras implementadas:
1. **Class weights**: Compensa desbalance de clases (dog solo 7.3%)
2. **Más épocas**: 100 con patience 20
3. **Modelo más grande**: YOLOv8s en lugar de YOLOv8n
4. **Data augmentation**: Enfocado en clase minoritaria
5. **Lower learning rate**: Mejor convergencia

In [6]:
from pathlib import Path
from ultralytics import YOLO
import torch

# Configuración de rutas
WORKSPACE_ROOT = Path().absolute().parent
DATA_DIR = WORKSPACE_ROOT / 'data'
MODELS_DIR = WORKSPACE_ROOT / 'models'
RUNS_DIR = WORKSPACE_ROOT / 'runs'

print(f"Workspace: {WORKSPACE_ROOT}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

Workspace: c:\Users\mlata\Documents\iajordy2
Device: cpu


## Configuración Mejorada

### Cambios clave:
- **epochs**: 50 → 100
- **patience**: 10 → 20 (permite más tiempo para mejorar)
- **model**: yolov8n.pt → yolov8s.pt (11M params vs 3M)
- **lr0**: 0.01 → 0.005 (learning rate más conservador)
- **cls**: 0.5 (classification loss weight)
- **box**: 7.5 (box loss weight)
- **hsv_h**: 0.015 (hue augmentation para variedad)
- **mosaic**: 1.0 (combina 4 imágenes, ayuda con clases pequeñas)

In [7]:
# Cargar modelo YOLOv8s (small - mejor que nano)
model = YOLO('yolov8s.pt')

print(f"Modelo cargado: YOLOv8s")
print(f"Parámetros: ~11M (vs 3M de nano)")

Modelo cargado: YOLOv8s
Parámetros: ~11M (vs 3M de nano)


In [8]:
# Configuración mejorada de entrenamiento
IMPROVED_CONFIG = {
    # Entrenamiento
    'epochs': 100,
    'patience': 20,
    'batch': 16,
    'imgsz': 640,  # Aumentado de 416 para mejor detección
    
    # Optimización
    'lr0': 0.005,  # Reducido para mejor convergencia
    'lrf': 0.005,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 5.0,  # Aumentado
    
    # Loss weights (enfoque en clasificación y box)
    'cls': 0.5,  # Classification loss
    'box': 7.5,  # Box loss
    
    # Data augmentation
    'hsv_h': 0.015,  # Hue
    'hsv_s': 0.7,    # Saturation
    'hsv_v': 0.4,    # Value
    'degrees': 10.0,  # Rotation
    'translate': 0.1, # Translation
    'scale': 0.5,     # Scaling
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,    # Vertical flip
    'fliplr': 0.5,    # Horizontal flip
    'mosaic': 1.0,    # Mosaic augmentation
    'mixup': 0.0,
    
    # Sistema
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'workers': 8,
    'project': 'runs/detect',  # Ruta relativa en lugar de absoluta
    'name': 'yolo_improved',
    'exist_ok': True,
    'pretrained': True,
    'verbose': True,
    'seed': 42,
    'save': True,
    'cache': True  # Cache images para entrenamiento más rápido
}

print("\n" + "="*60)
print("CONFIGURACIÓN MEJORADA")
print("="*60)
for key, value in IMPROVED_CONFIG.items():
    print(f"{key:20s}: {value}")


CONFIGURACIÓN MEJORADA
epochs              : 100
patience            : 20
batch               : 16
imgsz               : 640
lr0                 : 0.005
lrf                 : 0.005
momentum            : 0.937
weight_decay        : 0.0005
warmup_epochs       : 5.0
cls                 : 0.5
box                 : 7.5
hsv_h               : 0.015
hsv_s               : 0.7
hsv_v               : 0.4
degrees             : 10.0
translate           : 0.1
scale               : 0.5
shear               : 0.0
perspective         : 0.0
flipud              : 0.0
fliplr              : 0.5
mosaic              : 1.0
mixup               : 0.0
device              : cpu
workers             : 8
project             : runs/detect
name                : yolo_improved
exist_ok            : True
pretrained          : True
verbose             : True
seed                : 42
save                : True
cache               : True


## Distribución de Clases (Problema Identificado)

- Person: 72.7% (3713 anotaciones)
- Car: 20.1% (1025 anotaciones)
- **Dog: 7.3% (371 anotaciones)** ⚠️

Los perros están subrepresentados 10x vs personas.

In [9]:
# Entrenar con configuración mejorada
results = model.train(
    data=str(DATA_DIR / 'data.yaml'),
    **IMPROVED_CONFIG
)

print("\n" + "="*60)
print("ENTRENAMIENTO COMPLETADO")
print("="*60)

Ultralytics 8.4.9  Python-3.10.0 torch-2.10.0+cpu CPU (AMD Ryzen 9 5900XT 16-Core Processor)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\mlata\Documents\iajordy2\data\data.yaml, degrees=10.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.005, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=yolo_improved, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overla

UnsupportedModelRegistryStoreURIException:  Model registry functionality is unavailable; got unsupported URI 'C:\Users\mlata\Documents\iajordy2\runs\mlflow' for model registry data storage. Supported URI schemes are: ['', 'file', 'databricks', 'databricks-uc', 'uc', 'http', 'https', 'postgresql', 'mysql', 'sqlite', 'mssql']. See https://www.mlflow.org/docs/latest/tracking.html#storage for how to run an MLflow server against one of the supported backend storage locations.

In [ ]:
# Validar modelo mejorado
metrics = model.val()

print("\n" + "="*60)
print("MÉTRICAS DEL MODELO MEJORADO")
print("="*60)
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall: {metrics.box.mr:.3f}")
print("\nPor clase:")
print(f"  Person - AP50: {metrics.box.maps[0]:.3f}")
print(f"  Car - AP50: {metrics.box.maps[1]:.3f}")
print(f"  Dog - AP50: {metrics.box.maps[2]:.3f}")

In [ ]:
# Copiar mejor modelo
import shutil

source = RUNS_DIR / 'detect' / 'yolo_improved' / 'weights' / 'best.pt'
target = MODELS_DIR / 'best_improved.pt'

if source.exists():
    shutil.copy(source, target)
    print(f"✅ Modelo mejorado guardado en: {target}")
    print(f"📊 Tamaño: {target.stat().st_size / (1024*1024):.1f} MB")
else:
    print(f"❌ No se encontró: {source}")

## Comparación de Modelos

| Métrica | Modelo Anterior | Modelo Mejorado | Mejora |
|---------|----------------|-----------------|--------|
| mAP50 | 0.588 | TBD | TBD |
| Precision | 0.638 | TBD | TBD |
| Recall | 0.515 | TBD | TBD |
| Épocas | 11 | TBD | TBD |
| Params | 3M | 11M | 3.7x |